In [2]:
!pip install pymysql

  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/7c/4c/ad33b92b9864cbde84f259d5df035a6447f91891f5be77788e2a3892bce3/pymysql-1.1.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/45.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/45.3 kB ? eta -:--:--
   ------------------ --------------------- 20.5/45.3 kB 330.3 kB/s eta 0:00:01
   ------------------------------------ --- 41.0/45.3 kB 393.8 kB/s eta 0:00:01
   ---------------------------------------- 45.3/45.3 kB 321.7 kB/s eta 0:00:00


In [4]:
import os
from pathlib import Path
from dotenv import load_dotenv
import pymysql

In [6]:
env_file = Path(r"Location")  # <-- update this path to your actual location
load_dotenv(dotenv_path=env_file, override=True)

True

In [7]:
try:
    
    conn = pymysql.connect(
    host=os.getenv("MYSQL_HOST"),
    user=os.getenv("MYSQL_USER"),
    password=os.getenv("MYSQL_PASSWORD"),
    database=os.getenv("MYSQL_DB"),
    autocommit=True
)
    with conn.cursor() as cur:
        cur.execute("SELECT VERSION()")
        print("✅ Connected via PyMySQL. Server version:", cur.fetchone()[0])
except Exception as e:
    print("❌ Connection error:", e)
finally:
    try: conn.close()
    except: pass

✅ Connected via PyMySQL. Server version: 8.0.40


In [8]:
import pymysql
import time, random, json
from datetime import datetime, timedelta, timezone

# Connect once at the start
conn = pymysql.connect(
    host=os.getenv("MYSQL_HOST"),
    user=os.getenv("MYSQL_USER"),
    password=os.getenv("MYSQL_PASSWORD"),
    database=os.getenv("MYSQL_DB"),
    autocommit=True
)

# helper: create one fake order
def generate_order():
    order_id = f"ORD_{random.randint(10000, 99999)}"
    customer_id = f"CUST_{random.randint(1, 50)}"
    store_id = f"STORE_{random.randint(1, 10)}"
    order_value = round(random.uniform(200, 1500), 2)
    payment_method = random.choice(["upi", "card", "cod"])
    promised_eta_min = random.choice([20, 30, 45, 60])

    payload = {
        "order_id": order_id,
        "customer_id": customer_id,
        "store_id": store_id,
        "items": [{"sku": f"SKU{random.randint(100,999)}", "qty": random.randint(1,3)}],
        "order_value": order_value,
        "payment_method": payment_method,
        "promised_eta_min": promised_eta_min,
        "address": random.choice(["Bangalore","Mumbai","Delhi","Hyderabad"])
    }

    return dict(
        src_system="simulator",
        event_type="create",
        event_ts_utc=datetime.now(timezone.utc),
        ingest_ts_ist=datetime.now(timezone.utc) + timedelta(hours=5, minutes=30),
        order_id=order_id,
        customer_id=customer_id,
        store_id=store_id,
        order_value=order_value,
        payment_method=payment_method,
        promised_eta_min=promised_eta_min,
        payload_json=json.dumps(payload)
    )

# main loop: insert new orders
try:
    with conn.cursor() as cur:
        for i in range(50):   # change 5 → larger number for longer run
            order = generate_order()
            sql = """
            INSERT INTO raw_orders
            (src_system, event_type, event_ts_utc, ingest_ts_ist,
             order_id, customer_id, store_id, order_value, payment_method,
             promised_eta_min, payload_json)
            VALUES (%(src_system)s, %(event_type)s, %(event_ts_utc)s, %(ingest_ts_ist)s,
                    %(order_id)s, %(customer_id)s, %(store_id)s, %(order_value)s,
                    %(payment_method)s, %(promised_eta_min)s, %(payload_json)s);
            """
            cur.execute(sql, order)
            print(f"✅ Inserted {order['order_id']} at {order['ingest_ts_ist']}")
            time.sleep(2)  # wait 2 seconds between events
finally:
    conn.close()


✅ Inserted ORD_57380 at 2025-11-10 08:37:53.159261+00:00
✅ Inserted ORD_10763 at 2025-11-10 08:37:55.217092+00:00
✅ Inserted ORD_58857 at 2025-11-10 08:37:57.230022+00:00
✅ Inserted ORD_83795 at 2025-11-10 08:37:59.236994+00:00
✅ Inserted ORD_28761 at 2025-11-10 08:38:01.244620+00:00
✅ Inserted ORD_72506 at 2025-11-10 08:38:03.254989+00:00
✅ Inserted ORD_56642 at 2025-11-10 08:38:05.261198+00:00
✅ Inserted ORD_60471 at 2025-11-10 08:38:07.266176+00:00
✅ Inserted ORD_73418 at 2025-11-10 08:38:09.271545+00:00
✅ Inserted ORD_39887 at 2025-11-10 08:38:11.280795+00:00
✅ Inserted ORD_11866 at 2025-11-10 08:38:13.286511+00:00
✅ Inserted ORD_82988 at 2025-11-10 08:38:15.292605+00:00
✅ Inserted ORD_19526 at 2025-11-10 08:38:17.303304+00:00
✅ Inserted ORD_40374 at 2025-11-10 08:38:19.313014+00:00
✅ Inserted ORD_37351 at 2025-11-10 08:38:21.335903+00:00
✅ Inserted ORD_51993 at 2025-11-10 08:38:23.339355+00:00
✅ Inserted ORD_55720 at 2025-11-10 08:38:25.344717+00:00
✅ Inserted ORD_57479 at 2025-11

In [9]:
import pymysql
import random, json, time
from datetime import datetime, timedelta, timezone

conn = pymysql.connect(
    host=os.getenv("MYSQL_HOST"),
    user=os.getenv("MYSQL_USER"),
    password=os.getenv("MYSQL_PASSWORD"),
    database=os.getenv("MYSQL_DB"),
    autocommit=True
)
def generate_delivery(order_id):
    """Return one simulated delivery event for a given order."""
    delivery_id = f"DEL_{random.randint(10000,99999)}"
    courier_id = f"COURIER_{random.randint(1,20)}"
    hub_id = random.choice(["HUB_BLR","HUB_MUM","HUB_DEL","HUB_HYD"])
    status = random.choices(
        ["delivered","failed","returned"], weights=[0.8,0.15,0.05]
    )[0]

    now_utc = datetime.now(timezone.utc)
    promised_drop_utc = now_utc + timedelta(minutes=random.choice([20,30,45,60]))
    actual_drop_utc = promised_drop_utc + timedelta(minutes=random.randint(-10,15))

    base_cost = round(random.uniform(40,90),2)
    fuel_surcharge = round(base_cost * 0.1,2)
    other_cost = round(random.uniform(3,10),2)

    payload = {
        "delivery_id": delivery_id,
        "order_id": order_id,
        "courier_id": courier_id,
        "hub_id": hub_id,
        "status": status,
        "base_cost": base_cost,
        "fuel_surcharge": fuel_surcharge,
        "other_cost": other_cost,
        "currency": "INR"
    }

    return dict(
        src_system="simulator",
        event_type="drop",
        event_ts_utc=now_utc,
        ingest_ts_ist=now_utc + timedelta(hours=5, minutes=30),
        order_id=order_id,
        delivery_id=delivery_id,
        courier_id=courier_id,
        hub_id=hub_id,
        status=status,
        promised_drop_utc=promised_drop_utc,
        actual_drop_utc=actual_drop_utc,
        distance_km=round(random.uniform(2,15),2),
        base_cost=base_cost,
        fuel_surcharge=fuel_surcharge,
        other_cost=other_cost,
        currency="INR",
        payload_json=json.dumps(payload)
    )

try:
    with conn.cursor() as cur:
        # pick 5 random existing orders from raw_orders
        cur.execute("SELECT order_id FROM raw_orders ORDER BY RAND() LIMIT 5;")
        orders = [row[0] for row in cur.fetchall()]

        for order_id in orders:
            delivery = generate_delivery(order_id)
            sql = """
            INSERT INTO raw_deliveries (
                src_system, event_type, event_ts_utc, ingest_ts_ist,
                order_id, delivery_id, courier_id, hub_id, status,
                promised_drop_utc, actual_drop_utc, distance_km,
                base_cost, fuel_surcharge, other_cost, currency, payload_json
            )
            VALUES (%(src_system)s, %(event_type)s, %(event_ts_utc)s, %(ingest_ts_ist)s,
                    %(order_id)s, %(delivery_id)s, %(courier_id)s, %(hub_id)s, %(status)s,
                    %(promised_drop_utc)s, %(actual_drop_utc)s, %(distance_km)s,
                    %(base_cost)s, %(fuel_surcharge)s, %(other_cost)s, %(currency)s, %(payload_json)s);
            """
            cur.execute(sql, delivery)
            print(f"✅ Delivery {delivery['delivery_id']} inserted for Order {order_id}")
            time.sleep(2)

finally:
    conn.close()


✅ Delivery DEL_42532 inserted for Order ORD_37407
✅ Delivery DEL_92824 inserted for Order ORD_40374
✅ Delivery DEL_77372 inserted for Order ORD_53292
✅ Delivery DEL_62923 inserted for Order ORD_88234
✅ Delivery DEL_29417 inserted for Order ORD_31396
